In [1]:
import os
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

from src.core.config import get_settings

settings = get_settings()

INFO:CONC_MCPServer.src:Main package initialized


In [ ]:
# Load docs
from pathlib import Path

root_path = Path(os.getcwd()).joinpath("assets")
doc_paths = [
    root_path.joinpath("docs/test_rag.pdf"),
    root_path.joinpath("docs/test_rag.docx"),
]

docs = [] 
for file_path in doc_paths:

    try:
        if file_path.suffix == ".pdf":
            loader = PyPDFLoader(file_path)
        elif file_path.suffix == ".docx":
            loader = Docx2txtLoader(file_path)
        elif file_path.suffix == ".txt" or file_path.suffix == ".md":
            loader = TextLoader(file_path)
        else:
            print(f"Document type {file_path.suffix} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {file_path.name}: {e}")


# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

## Split documents

In [ ]:
# Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)

## Tokenize and load the documents to the vector store

In [ ]:
# Tokenize and load the documents to the vector store
vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [ ]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [ ]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

In [ ]:
url = settings.LYTIX_JUNIOR_FOUNDRY_URL
model = settings.LYTIX_JUNIOR_FOUNDRY_MODEL
model_version = settings.LYTIX_JUNIOR_FOUNDRY_MODEL_VERSION
print("\n\n")
print(f"URL: {url}")
print(f"MODEL: {model}")
print(f"MODEL VERSION: {model_version}")

In [ ]:
from langchain_openai import AzureChatOpenAI

llm_stream_new = AzureChatOpenAI(
    azure_endpoint=url,
    openai_api_version=model_version,
    model_name=model,
    openai_api_key=key,
    openai_api_type="azure",
    temperature=0.3,
    streaming=True,
)

prompt = "Tell me something about Azure"

for chunk in llm_stream_new.stream(prompt):
    print(chunk.content, end="", flush=True)